<a href="https://colab.research.google.com/github/KORAYANNIK/Regression_Cox_CCD_Screening/blob/main/Regression_de_Cox_Cyclic_Coordiante_Descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Donéées regression de Cox et calcul de la dérivé de la Negative Log-Likelihood
https://www.kaggle.com/datasets/wordsforthewise/lending-club?resource=download 
Donnée provenant de All Lending Club 2007 à 2018 sur [Kaggle](https://www.kaggle.com/datasets/wordsforthewise/lending-club?resource=download ) : En tout un million ̃données. Le data set retenu pour le training et le test est de 30 000 données.
$beta_i$



In [ ]:
my_seed = 0
import numpy as np
np.random.seed(my_seed)
import random
random.seed(my_seed)
import pandas as pd
from sklearn.preprocessing import StandardScaler,MinMaxScaler


precision=10**-15

In [ ]:
import re

In [ ]:
path="./drive/My Drive/Cox_Screening/loan.csv"
general_data=pd.read_csv(path,sep=',')
general_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29999 entries, 0 to 29998
Columns: 151 entries, id to settlement_term;
dtypes: float64(114), object(37)
memory usage: 34.6+ MB


Code du Cyclic Coordinate Bound 

In [ ]:
def CCB(Limite,fonction,indice,beta_prime): # fonction qui calcule les limites L et U de beta_prime(indice)
  
  L=np.copy(beta_prime)
  U=np.copy(beta_prime)
  #L[indice]=-1*abs(np.random.rand())
  #U[indice]=abs(np.random.rand())
  L[indice]=-1
  U[indice]=1

  lim=(U[indice])-(L[indice])
  f_L=fonction(L)
  f_U=fonction(U)
  while((lim<Limite) and ((f_L>=0) or (f_U<=0))):
    if (f_L>=0):
      L[indice]*=1.15
      f_L=fonction(L)
      
      
    if (f_U<=0):
      U[indice]*=1.15
      f_U=fonction(U)
    
    lim=U[indice]-L[indice]
    
  return(L,U)



Fonction du Cyclic Coordinate Descent

In [ ]:
def CCD(error_max,Iter_Max,Limite_ini,fonction,size):
  beta=np.zeros((size,1))
  error=fonction(beta)
  i=j=k=0
  while((i<Iter_Max) and abs(error)>error_max):
    #print(i)
    liste=list(range(size))
    random.shuffle(liste)
    for j in liste:
        L,U=CCB(Limite,fonction,j,beta)
        #print("iteration ",i,"indice",j,"L:",L,"U",U,"\n")
        ecart=U[j]-L[j]
        while((k<Iter_Max)and ecart>precision):
          beta[j]=(L[j]+U[j])/2
          error=fonction(beta)
          if error<0 :
            L=np.copy(beta)
          else : 
            U=np.copy(beta)
          ecart=U[j]-L[j]
          k+=1
    i+=1
  return (beta,abs(error))

In [ ]:
def CCDF(error_max,Iter_Max,Limite_ini,fonction,size,flags):
  beta=np.zeros((size,1))
  #error=fonction(beta)
  i=j=k=0
  error=10
  while((i<Iter_Max) and abs(error)>error_max):
    #print(i)
    liste=list(range(size))
    random.shuffle(liste)
    for j in liste:
        if flags[j]==1:
          break;
        fonc=lambda x : fonction(x,j)# derivé sur la j-ième composante de beta
        L,U=CCB(Limite,fonc,j,beta)
        print("calcul des bounds l'itération {}  de l'indice {} donne (L,U) :({},{})\n".format(i,j,L[j],U[j]))
        #print("iteration ",i,"indice",j,"L:",L,"U",U,"\n")
        ecart=U[j]-L[j]
        k=0
        while((k<Iter_Max)and ecart>precision):
          beta[j]=(L[j]+U[j])/2
          error=fonc(beta)
          print("calcul de l'erreur de l'itération {} de l'indice {} donne :{}\n".format(i,j,error))
          if error<0 :
            L=np.copy(beta)
          else : 
            U=np.copy(beta)
          ecart=U[j]-L[j]
          k+=1
          print("calcul du nouveau beta de  l'itération {} l'indice {} donne :{}\n".format(i,j,beta))
    i+=1
  return (beta)

Fonctions de test

In [ ]:
def f1(x):
  return 2*x[0]**3-8

In [ ]:
def f2(x):
  return x[0]**2+2*x[1]*2+1+2*x[1]*x[0]**1

In [ ]:
def f3(x):
  return x[0]+2*x[1]+2*x[0]*x[2]+1

In [ ]:
Limite=20
fonction=f2
indice=1
beta_prime=np.array([2,1],dtype=float)
CCB(Limite,fonction,indice,beta_prime)

(array([ 2., -1.]), array([2., 1.]))

In [ ]:
Limite=10
fonction=f1
indice=0
beta_prime=np.array([1],dtype=float)
CCB(Limite,fonction,indice,beta_prime)

(array([-1.]), array([1.74900625]))

In [ ]:
error_max=0.000001
Iter_Max=5000
Limite_ini=500
fonction=f2
size=2
CCD(error_max,Iter_Max,Limite_ini,fonction,size)

(array([[-9.35762087],
        [ 0.        ]]), array([88.56506841]))

In [ ]:
CCD(error_max,Iter_Max,Limite_ini,f3,3)

(array([[ 0.        ],
        [ 0.        ],
        [-9.35762087]]), array([1.]))

In [ ]:
general_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29999 entries, 0 to 29998
Columns: 151 entries, id to settlement_term;
dtypes: float64(114), object(37)
memory usage: 34.6+ MB


In [ ]:
general_data.select_dtypes(include="object").columns

Index(['id', 'term', 'grade', 'sub_grade', 'emp_title', 'emp_length',
       'home_ownership', 'verification_status', 'issue_d', 'loan_status',
       'pymnt_plan', 'url', 'purpose', 'title', 'zip_code', 'addr_state',
       'earliest_cr_line', 'initial_list_status', 'last_pymnt_d',
       'next_pymnt_d', 'last_credit_pull_d', 'application_type',
       'verification_status_joint', 'hardship_flag', 'hardship_type',
       'hardship_reason', 'hardship_status', 'hardship_start_date',
       'hardship_end_date', 'payment_plan_start_date', 'hardship_loan_status',
       'disbursement_method', 'debt_settlement_flag',
       'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
       'settlement_term;'],
      dtype='object')

In [ ]:
general_data[['last_pymnt_d','issue_d','loan_status']].head()

,last_pymnt_d,issue_d,loan_status
0,Jan-2019,Dec-2015,Fully Paid
1,Jun-2016,Dec-2015,Fully Paid
2,Jun-2017,Dec-2015,Fully Paid
3,Feb-2019,Dec-2015,Current
4,Jul-2016,Dec-2015,Fully Paid


In [ ]:
def date(date):
  months=["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
  times=[]
  for line in list(date):
    i=line[0]
    j=line[1]
    split1=str(i).split("-")
    split2=str(j).split("-")
    time=(months.index(split1[0])+1)*30+int(split1[1])*364
    time=time-((months.index(split2[0])+1)*30+int(split2[1])*364)
    times.append(time)
  return times


In [ ]:
features=["annual_inc","dti","emp_length","funded_amnt","grade","home_ownership","int_rate","loan_amnt","mort_acc","num_bc_sats","num_bc_tl","pub_rec_bankruptcies","revol_bal","term",'last_pymnt_d','issue_d','loan_status']
data=general_data[features]
data=data.drop_duplicates()
data=data.dropna()
times=date(data[['last_pymnt_d','issue_d']].values)
data["Time"]=times
data["Death"]=0
data.Death[data.loan_status=="Charged Off"]=1
data=data.drop(['last_pymnt_d','issue_d','loan_status'],axis=1)
#data.info()

<ipython-input-144-fe5bac9d79fb>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Death[data.loan_status=="Charged Off"]=1


In [ ]:
data.head()

,annual_inc,dti,emp_length,funded_amnt,grade,home_ownership,int_rate,loan_amnt,mort_acc,num_bc_sats,num_bc_tl,pub_rec_bankruptcies,revol_bal,term,Time,Death
0,55000.0,5.91,10+ years,3600.0,C,MORTGAGE,13.99,3600.0,1.0,2.0,5.0,0.0,2765.0,36 months,1126,0
1,65000.0,16.06,10+ years,24700.0,C,MORTGAGE,11.99,24700.0,4.0,13.0,17.0,0.0,21470.0,36 months,184,0
2,63000.0,10.78,10+ years,20000.0,B,MORTGAGE,10.78,20000.0,5.0,2.0,4.0,0.0,7869.0,60 months,548,0
3,110000.0,17.06,10+ years,35000.0,C,MORTGAGE,14.85,35000.0,1.0,8.0,10.0,0.0,7802.0,60 months,1156,0
4,104433.0,25.37,3 years,10400.0,F,MORTGAGE,22.45,10400.0,6.0,5.0,9.0,0.0,21929.0,60 months,214,0


In [ ]:
expr1="\d{1,2}"
temp=[]
for line in data.emp_length.values:
  temp.append(int(re.findall(expr1,str(line))[0]))
data.emp_length=temp
temp=[]
for line in data.term.values:
  temp.append(int(re.findall(expr1,str(line))[0]))
data.term=temp

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27920 entries, 0 to 29998
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   annual_inc            27920 non-null  float64
 1   dti                   27920 non-null  float64
 2   emp_length            27920 non-null  int64  
 3   funded_amnt           27920 non-null  float64
 4   grade                 27920 non-null  object 
 5   home_ownership        27920 non-null  object 
 6   int_rate              27920 non-null  float64
 7   loan_amnt             27920 non-null  float64
 8   mort_acc              27920 non-null  float64
 9   num_bc_sats           27920 non-null  float64
 10  num_bc_tl             27920 non-null  float64
 11  pub_rec_bankruptcies  27920 non-null  float64
 12  revol_bal             27920 non-null  float64
 13  term                  27920 non-null  int64  
 14  Time                  27920 non-null  int64  
 15  Death              

In [ ]:
data=pd.get_dummies(data,columns=["grade","home_ownership","term"],drop_first=False,dtype=int)
data_columns=['annual_inc', 'dti', 'emp_length', 'funded_amnt', 'int_rate',
       'loan_amnt', 'mort_acc', 'num_bc_sats', 'num_bc_tl',
       'pub_rec_bankruptcies', 'revol_bal', 'grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F', 'grade_G',
       'home_ownership_ANY', 'home_ownership_MORTGAGE', 'home_ownership_OWN',
       'home_ownership_RENT', 'term_36', 'term_60','Time', 'Death']
data=data[data_columns]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27920 entries, 0 to 29998
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   annual_inc               27920 non-null  float64
 1   dti                      27920 non-null  float64
 2   emp_length               27920 non-null  int64  
 3   funded_amnt              27920 non-null  float64
 4   int_rate                 27920 non-null  float64
 5   loan_amnt                27920 non-null  float64
 6   mort_acc                 27920 non-null  float64
 7   num_bc_sats              27920 non-null  float64
 8   num_bc_tl                27920 non-null  float64
 9   pub_rec_bankruptcies     27920 non-null  float64
 10  revol_bal                27920 non-null  float64
 11  grade_A                  27920 non-null  int64  
 12  grade_B                  27920 non-null  int64  
 13  grade_C                  27920 non-null  int64  
 14  grade_D               

In [ ]:
data_set=data.iloc[:200,:].copy()
Lasso=1

In [ ]:
feature_to_scale=["annual_inc","dti","emp_length","funded_amnt","int_rate","loan_amnt","mort_acc","num_bc_sats","num_bc_tl","pub_rec_bankruptcies","revol_bal"]
print(data_set[feature_to_scale].describe())
StandardScaling=MinMaxScaler().fit(data_set[feature_to_scale].values)#MinMaxScaler
data_set[feature_to_scale]=StandardScaling.transform(data_set[feature_to_scale])
data_set[feature_to_scale].describe()


          annual_inc         dti  emp_length   funded_amnt    int_rate  \
count     200.000000  200.000000  200.000000    200.000000  200.000000   
mean    73281.821050   19.965000    5.945000  15642.875000   12.473300   
std     32389.221427    8.575438    3.685088   8307.495472    4.287553   
min     15000.000000    3.300000    1.000000   1400.000000    5.320000   
25%     49000.000000   13.352500    2.000000   9056.250000    9.170000   
50%     70000.000000   18.665000    6.000000  15000.000000   12.435000   
75%     92000.000000   26.237500   10.000000  20550.000000   14.850000   
max    195000.000000   38.650000   10.000000  35000.000000   26.060000   

          loan_amnt    mort_acc  num_bc_sats   num_bc_tl  \
count    200.000000  200.000000   200.000000  200.000000   
mean   15642.875000    1.830000     4.990000    7.995000   
std     8307.495472    1.944222     2.807008    4.456938   
min     1400.000000    0.000000     0.000000    0.000000   
25%     9056.250000    0.000000  

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


,annual_inc,dti,emp_length,funded_amnt,int_rate,loan_amnt,mort_acc,num_bc_sats,num_bc_tl,pub_rec_bankruptcies,revol_bal
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.00000,200.000000
mean,0.323788,0.471429,0.549444,0.423895,0.344904,0.423895,0.228750,0.262632,0.347609,0.05250,0.124310
std,0.179940,0.242587,0.409454,0.247247,0.206729,0.247247,0.243028,0.147737,0.193780,0.16163,0.126185
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,0.188889,0.284371,0.111111,0.227865,0.185632,0.227865,0.000000,0.157895,0.217391,0.00000,0.052910
50%,0.305556,0.434653,0.555556,0.404762,0.343057,0.404762,0.125000,0.210526,0.304348,0.00000,0.086468
75%,0.427778,0.648868,1.000000,0.569940,0.459499,0.569940,0.375000,0.315789,0.478261,0.00000,0.154964
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000


In [ ]:
from numpy.core.fromnumeric import shape
def Derivate(betas):
  dataD=data_set.sort_values(by=["Death","Time"],axis=0)
  death_time=dataD[dataD.Death==1].Time.unique()
  grad=np.zeros_like(betas)
  
  
  for  k in range(len(betas)):
    l=0
    beta=betas[k]

    for t_i in death_time:
      sum1=0
      sum2=0
      sum3=0
      I_i=dataD[(dataD["Death"]==1) & (dataD["Time"]==t_i)].iloc[:,:-2].values #  I(i)
      #print("I_i shape",I_i.shape,"\n")
      R_i=dataD[(dataD["Death"]==0) | (dataD["Time"]>=t_i)].iloc[:,:-2].values #  R(i)
      #print("R_i shape",R_i.shape,"\n")
      d_i=I_i.shape[0]
      for x_s in I_i:
        sum1+=x_s[k]
      

      for x_j in R_i:
        #print("x_j",x_j.shape,"\n")
        sum2+=x_j[k]*np.exp(x_j@betas)
        sum3+=np.exp(x_j@betas)
      l+=sum1-(d_i/sum3)*sum2
    if beta>0:
      l+=Lasso
    else :
      l-=Lasso
    grad[k]=l
  return(grad)



In [ ]:
import math

In [ ]:
dataD_out=data_set.sort_values(by=["Death","Time"],axis=0)
death_time=dataD_out[dataD_out.Death==1].Time.unique()
R=[]
I=[]
d=[]
for t_i in death_time:
    I_i=dataD_out[(dataD_out["Death"]==1) & (dataD_out["Time"]==t_i)].iloc[:,:-2].values #  I(i)
    #print("I_i shape",I_i.shape,"\n")
    R_i=dataD_out[(dataD_out["Death"]==0) | (dataD_out["Time"]>=t_i)].iloc[:,:-2].values #  R(i)
    #print("R_i shape",R_i.shape,"\n")
    d_i=I_i.shape[0]
    I.append(I_i)
    R.append(R_i)
    d.append(d_i)

In [ ]:
def Derivate_unique(betas,indice):
  dataD=data_set.sort_values(by=["Death","Time"],axis=0)
  death_time=dataD[dataD.Death==1].Time.unique()
  grad=np.zeros(1)
  l=0
  beta=betas[indice]

  for t_i in death_time:
    sum1=0
    sum2=0
    sum3=0
    I_i=dataD[(dataD["Death"]==1) & (dataD["Time"]==t_i)].iloc[:,:-2].values #  I(i)
    #print("I_i shape",I_i.shape,"\n")
    R_i=dataD[(dataD["Death"]==0) | (dataD["Time"]>=t_i)].iloc[:,:-2].values #  R(i)
    #print("R_i shape",R_i.shape,"\n")
    d_i=I_i.shape[0]
    
    for x_s in I_i:
      sum1+=x_s[indice]
      

    for x_j in R_i:
      #print("x_j",x_j.shape,"\n")
      sum2+=x_j[indice]*np.exp(x_j@betas)
      sum3+=np.exp(x_j@betas)
    l+=sum1-(d_i/sum3)*sum2
  if beta>0:
    l+=Lasso
  else :
    l-=Lasso
  if (math.isinf(l)) or (np.isnan(l)):
    print("calcul de la dérivée de l'indice {} donne {}\n".format(indice,l))
  return(l)



In [ ]:
betas=np.ones(24)*1
Derivate(betas)


array([-3.29513944e+00,  6.23790605e-03, -8.33633806e+00, -3.87106256e+00,
        5.58066297e+00, -3.87106256e+00, -3.89583654e+00, -2.87086856e+00,
       -3.13679755e+00,  1.39027869e+00, -3.08212753e+00,  6.54450606e-01,
       -6.21023320e+00, -1.00023166e+00,  4.32485551e+00,  6.12261145e+00,
        2.10854730e+00,  1.00000000e+00,  1.00000000e+00, -9.88973845e+00,
        2.22429879e+00,  1.06654397e+01,  2.59869808e+00, -5.98698080e-01])

In [ ]:
Derivate_unique(betas,7)

-2.8708685605704556

In [ ]:
error_max=1
Iter_Max=10
Limite_ini=3
fonction=Derivate_unique
size=24
flags=np.zeros(size)


In [ ]:
array=CCDF(error_max,Iter_Max,Limite_ini,fonction,size,flags)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [-9.34750601]
 [-9.27670196]
 [ 0.        ]
 [ 0.        ]
 [-9.34750601]
 [ 0.        ]
 [ 5.3398004 ]
 [-9.34750601]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 5.3398004 ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]

calcul de l'erreur de l'itération 0 de l'indice 8 donne :[5.74145925]

calcul du nouveau beta de  l'itération 0 l'indice 8 donne :[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [-9.34750601]
 [-9.31716142]
 [ 0.        ]
 [ 0.        ]
 [-9.34750601]
 [ 0.        ]
 [ 5.3398004 ]
 [-9.34750601]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 5.3398004 ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]

calcul de l'erreur de l'itération 0 de l'indice 8 donne :[5.7458503]

calcul du nouveau beta de  l

In [ ]:
Derivate(array)

array([[ -3.0899947 ],
       [  8.53606742],
       [ 10.44444457],
       [-15.3727678 ],
       [  2.30472521],
       [-15.3727678 ],
       [  1.49999888],
       [  6.63157854],
       [  8.30434748],
       [  0.5       ],
       [  2.33862134],
       [  0.99999998],
       [  1.99999999],
       [-19.99999997],
       [  8.99999999],
       [  5.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [-16.99999998],
       [  2.99999998],
       [ 13.        ],
       [-17.        ],
       [ 17.        ]])

In [ ]:
dataD=data_set.sort_values(by=["Death","Time"],axis=0)
dataD[(dataD["Death"]==1) & (dataD["Time"]==244)].iloc[:,:-2].values.shape[0]

In [ ]:
dataD=data_set.sort_values(by=["Death","Time"],axis=0)
dataD[dataD.Death==1].Time.unique()

In [ ]:
I=dataD[(dataD["Death"]==0) | (dataD["Time"]>=1186)].iloc[:,:-2].values
I.shape

In [ ]:
dataD=data_set.sort_values(by=["Death","Time"],axis=0)
dataD[["Death","Time"]]

In [ ]:
dataD[dataD["Death"]==1].groupby(["Time"]).describe()